In [58]:
import glob
import pandas as pd
import numpy as np 
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import os
import torch
from torch import nn
import random

# Personnal Import 
from utilities import DataSet, get_batch
from dl_models.graph_conv import graphconv

## Data Description
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. 
    - Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.

- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

# On commence par simple
Restons sur de la prédiction.
On va sortir un modèle qui fait de la prédiction court-terme.
- Un model plutôt "image"
- Un model plutôt "graph"
Puis on les adaptera au fur et a mesure.
## Quoi prédire, pourquoi ?
- Prédiction Métro : Pour 'manager l'offre' (augmenter ou non la fréquence des transports), allouer des membre de staff, prévoir des évacuations d'urgence.  -> Toute la litérature cite ces 'scénarios', mais peu évaluent effectivement leur accuracy dessus. Ils donnent généralement des accuracy moyenne. 
## Focalisons sur un certains type d'événement.
- Match de sport ou concert en général c'est pas intéressant.
    - Parce que l'événement est prévu et récurrent, on sait le nombre de gens attendu. Donc l'offre de transport est adapté en amont, la fréquences des rames et TC alternatif sont prévu. Autre exemple, on pose des Vélo'v en masse dans une station fictive en sortie d'un match de foot. Alors c'est pas un événement intéressant. Biensur que si on entraine notre modèle avec ces événements assez récurrent et qu'on mets les bonnes informations, la prédiction court-terme pourra être améliorée. Mais à quoi bon ? 
- Match Sportif, Concert, ou Evenement de très grande ampleur : 
    - Concert de Ed Sheran, Finale d'Europa League : ce genre d'événement peut modifier fortement la demande en trafic sur tout le réseau, et il ne peut pas être correctement prévu. Les spectateur venant pour quelque jours, ils peuvent se donner rendez-vous a un endroit dans Lyon qui n'est pas forcément proche du Groupama. Exemple : des Holigans se donnent rendez-vous place des terreaux pour boire avant le match. Comment Anticiper l'augmentation de la demande en TC ? Pourquoi on a un intérêt à l'anticiper ? La seule raison que j'ai en tête c'est pour allouer des membre de staffs aux entrée /sorties pour la sécurité des usagers et maintenir la fréquence de l'offre. 

In [2]:
folder_path = 'data/Sub_Tram_11_2019_03_2020'
subway_paths, tramway_paths = sorted(glob.glob(os.path.join(folder_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(folder_path, "*df_tramway*.csv")))

In [3]:
def date_from_month_offset(n):
    n = int(n)
    reference_date = datetime(2019, 10, 1)
    years, months = divmod(n, 12)
    result_date = reference_date + timedelta(days=365 * years) + relativedelta(months=months)
    return f'{result_date.date().month}-{result_date.date().year}'
dic = {}
for list_paths,mode in zip([subway_paths,tramway_paths],['sub','tram']):
    dic[mode] = {}
    for csv_path in list_paths:
        month = csv_path.split('/')[2].split('_')[0]
        dic[mode][date_from_month_offset(month)] = csv_path 

In [4]:
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']
d = dates[0]

In [5]:
sub_nov = pd.read_csv(dic['sub'][d],index_col = 0)
sub_nov

,LIG_NUMERO_SAE,EMP_CEB,VAL_ARRET_CODE,CRS_SENS_TRAJET,VAL_DATE,Flow,COD_TRG,lane,LIB_STA_SIFO
0,0,VMMA-BON---1--1--1-,0,0,2019-11-20 04:36:00,3,BON,A,LAURENT BONNEVAY
1,0,VMMA-BON---1--1--1-,0,0,2019-11-20 04:57:00,7,BON,A,LAURENT BONNEVAY
2,0,VMMA-BON---1--1--1-,0,0,2019-11-20 05:00:00,2,BON,A,LAURENT BONNEVAY
3,0,VMMA-BON---1--1--1-,0,0,2019-11-20 05:03:00,3,BON,A,LAURENT BONNEVAY
4,0,VMMA-BON---1--1--1-,0,0,2019-11-20 05:12:00,1,BON,A,LAURENT BONNEVAY
...,...,...,...,...,...,...,...,...,...
2938824,304,VMMD-VAI---4--31-99,503,2,2019-11-29 16:03:00,1,VAI,D,GARE DE VAISE
2938825,304,VMMD-VAI---4--31-99,503,2,2019-11-29 16:06:00,1,VAI,D,GARE DE VAISE
2938826,304,VMMD-VAI---4--31-99,503,2,2019-11-29 16:48:00,1,VAI,D,GARE DE VAISE
2938827,304,VMMD-VAI---4--31-99,503,2,2019-11-29 17:54:00,1,VAI,D,GARE DE VAISE


In [6]:
tram_nov = pd.read_csv(dic['tram'][d],index_col = 0)
tram_nov

,LIG_NUMERO_SAE,EMP_CEB,VAL_ARRET_CODE,CRS_SENS_TRAJET,VAL_DATE,Flow,NOM_PNT,COD_LIG_CLI
0,530,VET--------832---10,32766,0,2019-11-06 17:03:00,2,Passage machine à laver UTT,T2
1,530,VET--------840---6-,32766,0,2019-11-22 05:39:00,1,Passage machine à laver UTT,T2
2,530,VET--------840---7-,32766,0,2019-11-22 05:39:00,1,Passage machine à laver UTT,T2
3,530,VET--------843---7-,32766,0,2019-11-04 04:27:00,1,Passage machine à laver UTT,T2
4,530,VET--------847---9-,32766,0,2019-11-16 09:27:00,2,Passage machine à laver UTT,T2
...,...,...,...,...,...,...,...,...
3034012,580,VET--------884---8-,46605,0,2019-11-16 19:27:00,4,Garage entrée Parc OL Ouest,TS
3034013,580,VET--------884---8-,46605,0,2019-11-16 19:33:00,2,Garage entrée Parc OL Ouest,TS
3034014,580,VET--------884---9-,46605,0,2019-11-16 19:30:00,3,Garage entrée Parc OL Ouest,TS
3034015,580,VET--------889---13,46605,0,2019-11-05 22:54:00,1,Garage entrée Parc OL Ouest,TS


## On va faire de la prédiction sur les validations Métro.  
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité.

In [37]:
sub_df = sub_nov[sub_nov['lane'] =='A'][['COD_TRG','Flow','VAL_DATE']]
sub_df = sub_df.groupby(['COD_TRG','VAL_DATE']).sum()
sub_df = sub_df.reset_index()
sub_df.VAL_DATE = pd.to_datetime(sub_df.VAL_DATE) 

# Reindex date
start,end = sub_df.VAL_DATE.iloc[0],sub_df.VAL_DATE.iloc[-1]
date_index = pd.date_range(start = start,end = end, freq = '3min')
sub_df = sub_df.pivot(index = 'VAL_DATE',columns = 'COD_TRG',values = 'Flow')
sub_df = sub_df.reindex(date_index).fillna(0)

#Reindex columns :
stations = ['PER','AMP','BEL','COR','HOT','FOC','MAS','CHA','REP','GRA','FLA','CUS','BON','SOI']
sub_df = sub_df[stations]
sub_df.head()

COD_TRG,PER,AMP,BEL,COR,HOT,FOC,MAS,CHA,REP,GRA,FLA,CUS,BON,SOI
2019-11-01 00:00:00,10.0,10.0,29.0,14.0,43.0,2.0,13.0,4.0,6.0,19.0,2.0,0.0,1.0,1.0
2019-11-01 00:03:00,28.0,2.0,18.0,32.0,41.0,3.0,7.0,16.0,2.0,6.0,0.0,2.0,2.0,1.0
2019-11-01 00:06:00,19.0,5.0,23.0,16.0,41.0,5.0,10.0,7.0,1.0,9.0,1.0,0.0,0.0,0.0
2019-11-01 00:09:00,19.0,3.0,29.0,8.0,64.0,0.0,12.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
2019-11-01 00:12:00,13.0,0.0,25.0,18.0,34.0,3.0,3.0,15.0,1.0,1.0,0.0,3.0,0.0,1.0


## Feature Vector
A définir pour chacune des stations

In [38]:
freq  ='3min'
freq_min = int(freq.split('min')[0])

time_step_per_hour = int(60/freq_min) #3min agg

historical_len = 7
Days = 1
Weeks = 1
step_ahead = 10


In [164]:
Ut = DataSet(sub_df,time_step_per_hour=time_step_per_hour)
norm_Ut = Ut.normalize()    # Normalize before getting the "Feature vector"  (or "Feature Tensor")
(X,Y,dates_verif) = norm_Ut.get_feature_vect(step_ahead,historical_len,Days,Weeks)
print('Feature vector shape: ',X.shape)   # Nb Sample, Nb Nodes, Sequence Length
dates_verif.head()

# Ut.df
# Ut.init_df
# Ut.time_step_per_hour
# Ut.mini
# Ut.maxi
# Ut.mean 
# norm_Ut.df.columns



Feature vector shape:  torch.Size([10560, 14, 9])


,t-3360,t-480,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t
3360,2019-11-01 00:00:00,2019-11-07 00:00:00,2019-11-07 23:12:00,2019-11-07 23:15:00,2019-11-07 23:18:00,2019-11-07 23:21:00,2019-11-07 23:24:00,2019-11-07 23:27:00,2019-11-07 23:30:00,2019-11-08 00:00:00
3361,2019-11-01 00:03:00,2019-11-07 00:03:00,2019-11-07 23:15:00,2019-11-07 23:18:00,2019-11-07 23:21:00,2019-11-07 23:24:00,2019-11-07 23:27:00,2019-11-07 23:30:00,2019-11-07 23:33:00,2019-11-08 00:03:00
3362,2019-11-01 00:06:00,2019-11-07 00:06:00,2019-11-07 23:18:00,2019-11-07 23:21:00,2019-11-07 23:24:00,2019-11-07 23:27:00,2019-11-07 23:30:00,2019-11-07 23:33:00,2019-11-07 23:36:00,2019-11-08 00:06:00
3363,2019-11-01 00:09:00,2019-11-07 00:09:00,2019-11-07 23:21:00,2019-11-07 23:24:00,2019-11-07 23:27:00,2019-11-07 23:30:00,2019-11-07 23:33:00,2019-11-07 23:36:00,2019-11-07 23:39:00,2019-11-08 00:09:00
3364,2019-11-01 00:12:00,2019-11-07 00:12:00,2019-11-07 23:24:00,2019-11-07 23:27:00,2019-11-07 23:30:00,2019-11-07 23:33:00,2019-11-07 23:36:00,2019-11-07 23:39:00,2019-11-07 23:42:00,2019-11-08 00:12:00


### Load adjacency Matrix: 

In [158]:
A_indep = torch.diag(torch.ones(len(Ut.df.columns)))   # Matrice d'adjacence identité, personne n'est connecté avec personne
A_Neighbors = torch.sum(torch.stack([torch.diag(torch.ones(len(Ut.df.columns)-abs(i)),i) for i in [-1,0,1]]),dim =0)   #Une seule ligne de métro, donc tri-diagonale
A_learnable = torch.nn.Parameter(torch.randn(len(Ut.df.columns),len(Ut.df.columns)),requires_grad=True)   #Matrice d'adjacence apprentissable

# Then convert into "Laplacian Matrix", or with "random_walk Matrix", or with another one ...
A_indep = 
A_Neighbors =
A_learnable = 

## Temporal Convolution

## Graph Convolution

In [125]:
class graphconv(nn.Module):
    def __init__(self,c_in,c_out,graph_conv_act_func,K = 2,enable_bias =True):
        super(graphconv,self).__init__()   # Demande a ce qu'on récupère les méthodes de la classe parent :  'nn.module'
        self.c_in = c_in
        self.c_out = c_out
        self.enable_bias = enable_bias
        self.graph_conv_act_func = graph_conv_act_func
        self.K = K
        if torch.cuda.is_available():
            self.weight = nn.Parameter(torch.cuda.FloatTensor(K,c_in,c_out))   # Initialize with wierd weight like 0e-30 or 1e38. Might not be totally adaptated ...
        else :
            self.weight = nn.Parameter(torch.FloatTensor(K,c_in,c_out))
        self.relu = nn.ReLU()

    def forward(self,x,gcnconv_matrix):
        B, C, L, N = x.shape
        n_mat =  gcnconv_matrix.shape[0]

        x = x.reshape(-1, self.c_in)  #[B, C_in, L, N] -> [BLN, C_in]
        x = torch.einsum('ab, cbd->cad',x,self.weight)   # [BLN,C_in], [K,C_in,C_out] -> [K,BLN,C_out]
        x = x.view(self.K, B*L,N,-1)  #[K,BLN,C_out] ->  [K,BL,N,C_out] 
        x = torch.einsum('ecab,ecbd->ecad',gcnconv_matrix,x)  #[n_adj,BL,N1,N2] ,[K,BL,N2,C_out]  -> [K,BL,N1,C_out] 

        if self.enable_bias:
            x = x + self.bias
        
        x = x.view().permute()
        x = self.relu(x)

        return(x)


In [274]:
x = torch.randn(2,1,3,4)
A_indep = torch.ones(3,3)

C_in = x.shape[1]
C_out = 8
weight = torch.FloatTensor(C_in,C_out)
weight = torch.Tensor([[1,2,3,4,5,6,7,8]])

adj_matrix =A_indep
# x.shape = [B,C,N2,L]
B,C_in,N2,L = x.shape
# A.shape = [n_adj,N1,N2]
N1,N2 = adj_matrix.shape
#A.shape = [n_adj,B,N1,N2]
stacked_adj = adj_matrix.repeat(1,B,1,1)


embedding = torch.einsum('bcnl,ch -> bhnl',x,weight)

In [309]:
class MGCN_conv(nn.Module):
    def __init__(self,c_in,c_out,n_adj=2,enable_bias =True):
        super(MGCN_conv,self).__init__()
        self.n_adj = n_adj    
        self.c_in = c_in
        self.c_out = c_out
        self.weight = torch.FloatTensor(n_adj,c_in,c_out)  #[C_in,C_out]   ou pour Einsum: [l,h]
        self.bias = torch.FloatTensor(c_out)
        self.relu = nn.ReLU()

    def forward(self,x,adj_matrix):
        # x.shape = [B,N2,L]
        B,N2,L = x.shape
        # A.shape = [n_adj,N1,N2]
        n_adj,N1,N2 = adj_matrix.shape

        #adj_matrix = torch.unsqueeze(adj_matrix,0) #Créer un nouvel axe 
        stacked_adj = adj_matrix.repeat(B,1,1) # Concat tout les "samples" (B batch, L tmeporal dimension) long du nouvel axe
        stacked_adj = stacked_adj.reshape(-1,N1,N2)  # Stack les n_adj adjacency matrix B*L fois     shape: [n_adj*B, N1,N2] et pour einsum : [b,p,n]

        embedding = torch.einsum('bnl,klh -> bknh',x,self.weight)   # Embedding sur les feature de chaque Noeud
        reshaped_embedding = embedding.reshape(-1,N2,self.c_out)    #Stack la dimension k sur l'axe 0  [n_adj*B,N2,C_out]  et pour einsum : [b,n,h]

        convoluted = torch.einsum('bpn,bnh -> bph',stacked_adj,reshaped_embedding) 
        convoluted = convoluted.reshape(self.n_adj,B,N1,self.c_out)      #Unstack la dimension 0, pour séparer les n_adj matrices d'adjacences

        convoluted = self.relu(convoluted + self.bias)

        return(convoluted,embedding,reshaped_embedding)

# Test : 
N1,N2 = 3,3
adj_matrix = torch.stack([torch.diag(torch.ones(N1)),torch.randn(N1,N2)],dim = 0)  # Matrice d'adjacence identité, personne n'est connecté avec personne
n_adj = adj_matrix.shape[0]

B = 4
L = 6
x = torch.randn(B,N1,L)

c_out = 16
gcn_model = MGCN_conv(L,c_out,n_adj)
AXW,embedding,reshaped_embedding = gcn_model(x,adj_matrix)

print("L'embedding a d'abord été opéré sur la dernière dimension (X*W, temporelle), Puis la convolution (A*(XW)) a sommmé les embedding de chacun des voisins (ou Noeud en lien avec le noeud tagret).")
print(f'X.shape: {x.shape}, AXW.shape:{AXW.shape}')

L'embedding a d'abord été opéré sur la dernière dimension (X*W, temporelle), Puis la convolution (A*(XW)) a sommmé les embedding de chacun des voisins (ou Noeud en lien avec le noeud tagret).
X.shape: torch.Size([4, 3, 6]), AXW.shape:torch.Size([2, 4, 3, 16])


In [312]:
embedding.shape,reshaped_embedding.shape

(torch.Size([4, 2, 3, 16]), torch.Size([8, 3, 16]))

In [320]:
#Bijecction entre un entier, et son label
def int2lab(int,n_adj=2,B=3):
    n = 1+(int-1)//B 
    b = int-(n-1)*B
    return(f'n{n}_b{b}')

def lab2int(lab,n_adj=2,B=3):
    nb = lab.split('_')
    n,b = int(nb[0][1:]),int(nb[1][1:])
    return((n-1)*B+b)

for k in range(1,15):
    print(int2lab(k),lab2int(int2lab(k)))

n1_b1 1
n1_b2 2
n1_b3 3
n2_b1 4
n2_b2 5
n2_b3 6
n3_b1 7
n3_b2 8
n3_b3 9
n4_b1 10
n4_b2 11
n4_b3 12
n5_b1 13
n5_b2 14


## Représentation des numéro des cellules du Tensor
Permet de faire des affichages graphique, et de s'assurer que les ".reshape" font ce que l'on souhaite

In [337]:
def int2lab(integer, n_adj=2, B=3, C=4, N=5, L=6):
    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    l = 1 + ((integer - 1) % L)
    remaining = (integer - 1) // L
    n = 1 + (remaining % N)
    remaining //= N
    c = 1 + (remaining % C)
    remaining //= C
    b = 1 + (remaining % B)
    remaining //= B
    n_adj = 1 + remaining % n_adj

    return f'adj{n_adj}_b{b}_c{c}_n{n}_l{l}'

def lab2int(label, n_adj=2, B=3, C=4, N=5, L=6):
    split_label = label.split('_')
    n_adj = int(split_label[0][3:])
    b = int(split_label[1][1:])
    c = int(split_label[2][1:])
    n = int(split_label[3][1:])
    l = int(split_label[4][1:])

    N_adj_L = N * L
    N_adj_C_L = C * N_adj_L
    N_adj_B_C_L = B * N_adj_C_L

    integer = ((l - 1) + (n - 1) * L + (c - 1) * N * L + (b - 1) * C * N * L + (n_adj - 1) * B * C * N * L) + 1
    return integer


for k in range(1,400):
    print(int2lab(k),lab2int(int2lab(k)))

adj1_b1_c1_n1_l1 1
adj1_b1_c1_n1_l2 2
adj1_b1_c1_n1_l3 3
adj1_b1_c1_n1_l4 4
adj1_b1_c1_n1_l5 5
adj1_b1_c1_n1_l6 6
adj1_b1_c1_n2_l1 7
adj1_b1_c1_n2_l2 8
adj1_b1_c1_n2_l3 9
adj1_b1_c1_n2_l4 10
adj1_b1_c1_n2_l5 11
adj1_b1_c1_n2_l6 12
adj1_b1_c1_n3_l1 13
adj1_b1_c1_n3_l2 14
adj1_b1_c1_n3_l3 15
adj1_b1_c1_n3_l4 16
adj1_b1_c1_n3_l5 17
adj1_b1_c1_n3_l6 18
adj1_b1_c1_n4_l1 19
adj1_b1_c1_n4_l2 20
adj1_b1_c1_n4_l3 21
adj1_b1_c1_n4_l4 22
adj1_b1_c1_n4_l5 23
adj1_b1_c1_n4_l6 24
adj1_b1_c1_n5_l1 25
adj1_b1_c1_n5_l2 26
adj1_b1_c1_n5_l3 27
adj1_b1_c1_n5_l4 28
adj1_b1_c1_n5_l5 29
adj1_b1_c1_n5_l6 30
adj1_b1_c2_n1_l1 31
adj1_b1_c2_n1_l2 32
adj1_b1_c2_n1_l3 33
adj1_b1_c2_n1_l4 34
adj1_b1_c2_n1_l5 35
adj1_b1_c2_n1_l6 36
adj1_b1_c2_n2_l1 37
adj1_b1_c2_n2_l2 38
adj1_b1_c2_n2_l3 39
adj1_b1_c2_n2_l4 40
adj1_b1_c2_n2_l5 41
adj1_b1_c2_n2_l6 42
adj1_b1_c2_n3_l1 43
adj1_b1_c2_n3_l2 44
adj1_b1_c2_n3_l3 45
adj1_b1_c2_n3_l4 46
adj1_b1_c2_n3_l5 47
adj1_b1_c2_n3_l6 48
adj1_b1_c2_n4_l1 49
adj1_b1_c2_n4_l2 50
adj1_b1_c

In [332]:
label = int2lab(k)
label.split('_')

['n1', 'b1', 'c1', 'adj1', 'l1']

In [ ]:
class GCN_conv(nn.Module):
    def __init__(self,c_in,c_out,graph_conv_act_func,K=1,enable_bias =True):
        super(GCN_conv,self).__init__()

    
    def forward(self,x,adj_matrix):
        # x.shape = [B,C,N2,L]
        B,C,N2,L = x.shape
        # A.shape = [n_adj,N1,N2]
        n_adj,N1,N2 = adj_matrix.shape
        #A.shape = [n_adj,B,N1,N2]
        adj_matrix = torch.unsqueeze(adj_matrix,0) #Créer un nouvel axe 
        stacked_adj = adj_matrix.repeat(B*L,1,1,1) # Concat tout les "samples" (B batch, L tmeporal dimension) long du nouvel axe
        stacked_adj = stacked_adj.reshape(-1,N1,N2)  # Stack les n_adj adjacency matrix B*L fois

        embedding = torch.einsum('bcnl,ch -> bhnl',x,self.weight)   # Embedding Spatial le long de C_in
        embedding = embedding.permute(0,3,2,1)  # [B,L,N2,C_out]
        embedding = embedding.reshape(-1,N2,C_out)  # [BL,N2,C_out]

        convolution = torch.einsum('kbpn,bhnl -> kbpl',stacked_adj,embedding)









In [285]:
n_adj_matrix = adj_matrix.repeat(2,1,1)
n_adj,N1,N2 = n_adj_matrix.shape
#A.shape = [n_adj,B,N1,N2]
stacked_adj = n_adj_matrix.repeat(B*L,1,1) # [n_adj*B, N1,N2]  -> Concat tout les "samples" long de l'axe 0
stacked_adj.shape,n_adj_matrix.shape

(torch.Size([16, 3, 3]), torch.Size([2, 3, 3]))

In [192]:
x = X.unsqueeze(1) #add the channel dimension (here, only "flow')
print(f'X shape: {x.shape}')
x_b = x[:32]
print(f'x_b shape: {x_b.shape}')

GCN = graphconv(c_in = x.shape[1], c_out = 64, K=2, graph_conv_act_func = 'relu',enable_bias=True)  # K =2 dans MRGNN car considère Pattern et Adj matrix
B, C, N, L = x_b.shape
K = 1
c_out = 64
n_mat =  A_indep.shape[0]

print(f'B,C,N,L : [{B},{C},{N},{L}]')

X shape: torch.Size([10560, 1, 14, 9])
x_b shape: torch.Size([32, 1, 14, 9])
B,C,N,L : [32,1,14,9]


## GCN détaillé 

In [255]:
weight = nn.Parameter(torch.FloatTensor(K,C,c_out))
bias = nn.Parameter(torch.FloatTensor(c_out))
print(f'x_b: {x_b.shape}, W: {weight.shape}, Cause there are {K} adjacency matrices (K), {C} C_in and {c_out} C_out')
x_b = x_b.reshape(-1, C)  #[B, C_in, L, N] -> [BLN, C_in]
print(f'reshaped x_b: {x_b.shape}, Cause we have to flatten x_b along the Channel axis, and then pass through a SPATIAL embedding (Linear layer, on C_in of each sample,nodes,historical element)')

# Embedding on C_in:  X*W
embedd_c_in = torch.einsum('ab, cbd->cad',x_b,weight)   # [BLN,C_in], [K,C_in,C_out] -> [K,BLN,C_out]n  Propose K embedding de C_in
print('shape of reshaped_xb after K embedding on C_in: ',embedd_c_in.shape)
reshaped_embedd_c_in = embedd_c_in.view(K, B*L,N,-1)  #[K,BLN,C_out] ->  [K,BL,N,C_out] 
print('Embedded feature vect reshaped: ',reshaped_embedd_c_in.shape, '\n')

# Concat Adj Matrix 
batched_adj_matrix = A_indep.repeat(1,B*L,1,1)
print('Adjacency matrix A_indep: ',A_indep.shape,'batched Multi adj_matrix: ',batched_adj_matrix.shape)

# Convolution A*(XW)
convolutionned = torch.einsum('ecab,ecbd->ecad',batched_adj_matrix,reshaped_embedd_c_in)  #[K,BL,N1,N2] ,[K,BL,N2,C_out]  -> [K,BL,N1,C_out] 
print('Convolution A*(XW): ',convolutionned.shape)

#Add bias: 
convolutionned_n_biased = convolutionned + bias

# Reshape and Permute: 
H =convolutionned_n_biased.view(K,B,L,-1,c_out).permute(0,1,4,3,2)
print(f'Reshaped convolution output: {H.shape}')

x_b: torch.Size([4032, 1]), W: torch.Size([1, 1, 64]), Cause there are 1 adjacency matrices (K), 1 C_in and 64 C_out
reshaped x_b: torch.Size([4032, 1]), Cause we have to flatten x_b along the Channel axis, and then pass through a SPATIAL embedding (Linear layer, on C_in of each sample,nodes,historical element)
shape of reshaped_xb after K embedding on C_in:  torch.Size([1, 4032, 64])
Embedded feature vect reshaped:  torch.Size([1, 288, 14, 64]) 

Adjacency matrix A_indep:  torch.Size([14, 14]) batched Multi adj_matrix:  torch.Size([1, 288, 14, 14])
Convolution A*(XW):  torch.Size([1, 288, 14, 64])
Reshaped convolution output: torch.Size([1, 32, 64, 14, 9])


In [249]:
permuted_H = output_H.permute(0, 1, 3, 4, 2)
print(permuted_H.shape)

new_c_in = permuted_H.shape[-1]
first_attn = permuted_H.reshape(K,-1,new_c_in)
print(first_attn.shape)  # [K, B*L*N, C_in']

attn_weight = nn.Linear(new_c_in,1)
softmax = nn.Softmax(-1)  #SoftMax on the last dimension 

#Coefficient d'attention 
attn = attn_weight(first_attn)    # "Embedding" du spatial channel 
attn = attn.permute(1,2,0)  # Permute pour avoir la nombre d'adjacency matrix en dernière dimension
attn = softmax(attn)  # Coefficient d'attention pour chaque Matrices d'adjacence et Embedding spatial assicié  (Ici 1 seule matrice d'adjacence)

torch.Size([1, 32, 9, 14, 64])
torch.Size([1, 4032, 64])


In [248]:
(attn == 1).sum()

tensor(4032)

In [229]:
a = torch.randn(2,3,4)
a

tensor([[[ 1.1155,  0.2000, -0.2143, -0.2138],
         [-0.6428, -1.0331, -0.2202, -1.8725],
         [-0.9164,  0.0879, -1.4340,  0.1705]],

        [[ 0.3603,  0.1547,  0.0509, -0.0657],
         [ 0.7353,  1.4075, -1.5137,  1.6018],
         [ 0.7461, -0.3099, -0.6415, -0.4405]]])

In [ ]:
if self.enable_bias:
    x = x + self.bias

x = x.view().permute()
x = self.relu(x)